In [1]:
#Remarks
#check coefficients against equations
#be careful of copying and reassign values in numpy array
%reset -f
import h5py
import os
import numpy as np

#modify user inputs below
mesh_prefix='sphere'
outfile='sphere_s61.hdf5'
pot_file='none.pot'
freq=[0.01,0.1] #simulating frequencies

sigma_static=0.01 #surface charge density [C/m^2]
rel_perm_ebulk=78.5 #relative permittivity of bulk electrolyte [F/m]
rel_perm_solid=4.5 #relative permittivity of solid particle [F/m]
diffusion_ebulk=2*1e-6**2/1e-3 #diffusion coefficient of water at 20 deg. C is 2*(1e-6*m)^2/(1e-3*s)
diffusion_stern=2*1e-6**2/1e-2 #diffusion coefficient in Stern layer [m^2/s]
mobility_ebulk=5e-8 #mobility of sodium cation in bulk electrolyte [m^2/(Vs)]
mobility_stern=5e-9 #mobility in Stern layer [m^2/(Vs)]

temp_K=298 #room/ambient temperature [K]
cinf=100 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #ion concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
#adjust mesh by radius
print('Adjusting mesh by radius (will be removed in the future)')
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

# print(min(dist1[mask]),max(dist1[mask]))
# print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]

#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)
print('')

#define computational volume and boundaries
mask_e=zones>=0 #True for elements used in computation
mask_n=(node_flags==1)|(node_flags==2) #True for nodes on dirichlet b.c.
mask_f=face_flags==20 #True for faces on third kind b.c.

#load static potential and potential gradient
print('Reading potential %s'%pot_file)
if os.path.isfile(pot_file):
    u=np.genfromtxt(potfile,skip_header=1,usecols=1)
    grad_u=np.genfromtxt(potfile,skip_header=1,usecols=(2,3,4))
else:
    u=np.ones(nelem)
    grad_u=np.zeros((nelem,3))

#set material properties
c=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
alpha_x=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
alpha_y=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
alpha_z=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]

gamma_x=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
gamma_y=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
gamma_z=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]

beta_x=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
beta_y=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]
beta_z=[np.zeros(nelem),np.zeros((nelem,3,3)),np.zeros(nface)]

a=[np.zeros(nelem),np.zeros((nelem,3,3),dtype=complex),np.zeros(nface,dtype=complex)]
f=[np.zeros(nelem),np.zeros((nelem,3)),np.zeros(nface)]
gs=[np.zeros(nface),np.zeros((nface,3)),np.zeros(0)]
qs=[np.zeros(nface),np.zeros((nface,3,3)),np.zeros(0)]
sn=[np.zeros(nnode),np.zeros((nnode,3)),np.zeros(0)]

print('Setting PDE coefficients for material_0 (static part)')
c[0][zones>=3]=perm0*rel_perm_ebulk
c[0][zones<=2]=perm0*rel_perm_solid
gs[0][face_flags==20]=sigma_static

print('Setting PDE coefficients for material_1 (frequency volume part)')
c[1][zones>=3,0,0]=diffusion_ebulk
c[1][zones>=3,1,1]=diffusion_ebulk
c[1][zones>=3,2,2]=perm0*rel_perm_ebulk
c[1][zones<=2,2,2]=perm0*rel_perm_solid
c[1][zones>=3,0,2]=-mobility_ebulk*cinf*np.exp(echarge/kB/temp_K*u[zones>=3])
c[1][zones>=3,1,2]=mobility_ebulk*cinf*np.exp(-echarge/kB/temp_K*u[zones>=3])

alpha_x[1][zones>=3,0,0]=-mobility_ebulk*grad_u[zones>=3,0]
alpha_y[1][zones>=3,0,0]=-mobility_ebulk*grad_u[zones>=3,1]
alpha_z[1][zones>=3,0,0]=-mobility_ebulk*grad_u[zones>=3,2]

alpha_x[1][zones>=3,1,1]=mobility_ebulk*grad_u[zones>=3,0]
alpha_y[1][zones>=3,1,1]=mobility_ebulk*grad_u[zones>=3,1]
alpha_z[1][zones>=3,1,1]=mobility_ebulk*grad_u[zones>=3,2]

a[1][zones>=3,0,0]=1j*0.1 #angular frequency as 0.1
a[1][zones>=3,1,1]=1j*0.1 #angular frequency as 0.1
a[1][zones>=3,2,0]=echarge
a[1][zones>=3,2,1]=-echarge

gs[1][mask_f,2]=0.005 #initial surface charge density [C/m^2]
sn[1][mask_n,2]=0.1 #external potential field

print('Setting PDE coefficients for material_2 (frequency surface part)')
c[2][face_flags==20]=diffusion_stern
a[2][face_flags==20]=1j*0.1 #angular frequency as 0.1

print('Saving coefficients to %s'%outfile)
with h5py.File(outfile,'w') as file:    
    file.create_dataset('mesh/elements',data=elements,dtype='float32')
    file.create_dataset('mesh/nodes',data=nodes,dtype='float32')
    file.create_dataset('mesh/faces',data=faces,dtype='float32')
    
    file.create_dataset('mesh/zones',data=zones,dtype='int32')
    file.create_dataset('mesh/node_flags',data=node_flags,dtype='int32')
    file.create_dataset('mesh/face_flags',data=face_flags,dtype='int32')
        
    file.create_dataset('mesh/mask_elements',data=mask_e,dtype='bool')
    file.create_dataset('mesh/mask_nodes_first_kind_bc',data=mask_n,dtype='bool')
    file.create_dataset('mesh/mask_faces_third_kind_bc',data=mask_f,dtype='bool')
    
#     file.create_dataset('physics/echarge',data=echarge,dtype='float')
#     file.create_dataset('physics/perm0',data=perm0,dtype='float')
#     file.create_dataset('physics/kA',data=kA,dtype='float')
#     file.create_dataset('physics/kB',data=kB,dtype='float')
    
    file.create_dataset('physics/rel_perm_ebulk',data=rel_perm_ebulk,dtype='float')
    file.create_dataset('physics/rel_perm_solid',data=rel_perm_solid,dtype='float')
    file.create_dataset('physics/diffusion_ebulk',data=diffusion_ebulk,dtype='float')
    file.create_dataset('physics/diffusion_stern',data=diffusion_stern,dtype='float')
    file.create_dataset('physics/mobility_ebulk',data=mobility_ebulk,dtype='float')
    file.create_dataset('physics/mobility_stern',data=mobility_stern,dtype='float')
    file.create_dataset('physics/temperature',data=temp_K,dtype='float')
    file.create_dataset('physics/ion_conc_inf',data=cinf,dtype='float')
    file.create_dataset('physics/ion_val',data=zval,dtype='float')
    
    #create subgroups for frequencies
    for i in range(len(freq)):
        file.create_group('frequency_%.6e'%freq[i])
    
#     #save real part of PDE coefficients
#     for i in range(3):
#         file.create_dataset('material_%d/cx/real'%i,data=np.real(c[i]),dtype='float32')
#         file.create_dataset('material_%d/cy/real'%i,data=np.real(c[i]),dtype='float32')
#         file.create_dataset('material_%d/cz/real'%i,data=np.real(c[i]),dtype='float32')
        
#         file.create_dataset('material_%d/alpha_x/real'%i,data=np.real(alpha_x[i]),dtype='float32')
#         file.create_dataset('material_%d/alpha_y/real'%i,data=np.real(alpha_y[i]),dtype='float32')
#         file.create_dataset('material_%d/alpha_z/real'%i,data=np.real(alpha_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/gamma_x/real'%i,data=np.real(gamma_x[i]),dtype='float32')
#         file.create_dataset('material_%d/gamma_y/real'%i,data=np.real(gamma_y[i]),dtype='float32')
#         file.create_dataset('material_%d/gamma_z/real'%i,data=np.real(gamma_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/beta_x/real'%i,data=np.real(beta_x[i]),dtype='float32')
#         file.create_dataset('material_%d/beta_y/real'%i,data=np.real(beta_y[i]),dtype='float32')
#         file.create_dataset('material_%d/beta_z/real'%i,data=np.real(beta_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/a/real'%i,data=np.real(a[i]),dtype='float32')
#         file.create_dataset('material_%d/f/real'%i,data=np.real(f[i]),dtype='float32')
#         file.create_dataset('material_%d/gs/real'%i,data=np.real(gs[i]),dtype='float32')
#         file.create_dataset('material_%d/qs/real'%i,data=np.real(qs[i]),dtype='float32')
#         file.create_dataset('material_%d/sn/real'%i,data=np.real(sn[i]),dtype='float32')
        
#     #save imaginary part of PDE coefficients
#     for i in range(3):
#         file.create_dataset('material_%d/cx/imag'%i,data=np.imag(c[i]),dtype='float32')
#         file.create_dataset('material_%d/cy/imag'%i,data=np.imag(c[i]),dtype='float32')
#         file.create_dataset('material_%d/cz/imag'%i,data=np.imag(c[i]),dtype='float32')
        
#         file.create_dataset('material_%d/alpha_x/imag'%i,data=np.imag(alpha_x[i]),dtype='float32')
#         file.create_dataset('material_%d/alpha_y/imag'%i,data=np.imag(alpha_y[i]),dtype='float32')
#         file.create_dataset('material_%d/alpha_z/imag'%i,data=np.imag(alpha_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/gamma_x/imag'%i,data=np.imag(gamma_x[i]),dtype='float32')
#         file.create_dataset('material_%d/gamma_y/imag'%i,data=np.imag(gamma_y[i]),dtype='float32')
#         file.create_dataset('material_%d/gamma_z/imag'%i,data=np.imag(gamma_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/beta_x/imag'%i,data=np.imag(beta_x[i]),dtype='float32')
#         file.create_dataset('material_%d/beta_y/imag'%i,data=np.imag(beta_y[i]),dtype='float32')
#         file.create_dataset('material_%d/beta_z/imag'%i,data=np.imag(beta_z[i]),dtype='float32')
        
#         file.create_dataset('material_%d/a/imag'%i,data=np.imag(a[i]),dtype='float32')
#         file.create_dataset('material_%d/f/imag'%i,data=np.imag(f[i]),dtype='float32')
#         file.create_dataset('material_%d/gs/imag'%i,data=np.imag(gs[i]),dtype='float32')
#         file.create_dataset('material_%d/qs/imag'%i,data=np.imag(qs[i]),dtype='float32')
#         file.create_dataset('material_%d/sn/imag'%i,data=np.imag(sn[i]),dtype='float32')

print('Done')

Reading sphere.1.node
Reading sphere.1.ele
Reading sphere.1.face
Adjusting mesh by radius (will be removed in the future)
THE NUMBER OF NODES IS: 24157
THE NUMBER OF ELEMENTS IS: 135851
THE NUMBER OF FACES IS: 17578

Reading potential none.pot
Setting PDE coefficients for material_0 (static part)
Setting PDE coefficients for material_1 (frequency volume part)
Setting PDE coefficients for material_2 (frequency surface part)
Saving coefficients to sphere_s61.hdf5
Done


In [3]:
import h5py
import numpy as np

f=h5py.File('sphere_s61.hdf5','r')
print(f.keys())
print(f['mesh'].keys())
print(f['physics'].keys())
f.close()

<KeysViewHDF5 ['frequency_1.000000e-01', 'frequency_1.000000e-02', 'mesh', 'physics']>
<KeysViewHDF5 ['elements', 'face_flags', 'faces', 'mask_elements', 'mask_faces_third_kind_bc', 'mask_nodes_first_kind_bc', 'node_flags', 'nodes', 'zones']>
<KeysViewHDF5 ['diffusion_ebulk', 'diffusion_stern', 'ion_conc_inf', 'ion_val', 'mobility_ebulk', 'mobility_stern', 'rel_perm_ebulk', 'rel_perm_solid', 'temperature']>
